In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import math
import copy

In [2]:
import torch
import torch.nn as nn
from torch.utils import data

import torchvision.transforms as transforms
import torchvision.models as models

## Image to Tensor Conversions

In [5]:
imsize = 256

def image_to_tensor(file_path):
    image = Image.open(file_path)
    ten_image = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])(image)
    ten_image.unsqueeze_(0) # adds batch size = 1
    return ten_image

In [12]:
set_folder = "Set1"

content = image_to_tensor("UpdatedSets/"+set_folder+"/content.jpg")
style = image_to_tensor("UpdatedSets/"+set_folder+"/style.png")

In [13]:
if content.size() != style.size():
    print("Please execute Processing the Data.ipynb on the dataset")